<a href="https://colab.research.google.com/github/EricGoulart/BibMon/blob/main/INF1032_Projeto_Amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
import re
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix, classification_report, ConfusionMatrixDisplay
)
from sklearn.linear_model import LogisticRegression

try:
    import kagglehub
except ImportError as e:
    raise ImportError(
        "Este script depende do pacote 'kagglehub'. Instale com: pip install kagglehub"
    ) from e

from pathlib import Path

# Baixa a versão mais recente do dataset e obtém a pasta local em cache
DATASET_SLUG = "ikramshah512/amazon-products-sales-dataset-42k-items-2025"
DATA_DIR = Path(kagglehub.dataset_download(DATASET_SLUG))
print("Path to dataset files:", DATA_DIR)

def escolher_uncleaned_csv(data_dir: Path) -> Path:
    """
    Seleciona o arquivo 'uncleaned' do dataset.
    Regras:
      1) Tenta nome exato: 'amazon_products_sales_data_uncleaned.csv'
      2) Se não achar, procura qualquer *.csv com 'unclean' no nome (case-insensitive)
         e escolhe o maior (normalmente é o principal).
    """
    # 1) Nome exato
    exatos = list(data_dir.rglob("amazon_products_sales_data_uncleaned.csv"))
    if exatos:
        print(f"CSV (nome exato): {exatos[0]}")
        return exatos[0]

    # 2) Fallback por padrão '*unclean*'
    candidatos = [p for p in data_dir.rglob("*.csv") if "unclean" in p.name.lower()]
    if candidatos:
        escolhido = max(candidatos, key=lambda p: p.stat().st_size)
        print(f"CSV (padrão '*unclean*'): {escolhido}")
        return escolhido

    raise FileNotFoundError(
        f"Não encontrei o arquivo 'uncleaned' dentro de {data_dir}."
    )

# Caminho do arquivo selecionado dentro do dataset baixado
PATH = str(escolher_uncleaned_csv(DATA_DIR))

# Caminho do arquivo (ajuste se necessário)
# PATH = 'amazon_products_sales_data_uncleaned.csv'

# Opções de exibição (facilitam ler no terminal)
pd.set_option('display.max_columns', 120)
pd.set_option('display.width', 180)

Using Colab cache for faster access to the 'amazon-products-sales-dataset-42k-items-2025' dataset.
Path to dataset files: /kaggle/input/amazon-products-sales-dataset-42k-items-2025
CSV (nome exato): /kaggle/input/amazon-products-sales-dataset-42k-items-2025/amazon_products_sales_data_uncleaned.csv


---
## ETAPA 1 — Leitura do CSV + Visão Geral (info)
**O que aconteceu:** Lemos o CSV para um DataFrame pandas e imprimimos `df.info()`.

**O que está acontecendo:** `df.info()` mostra a contagem de não-nulos e os tipos de cada coluna, confirmando o tamanho do dataset.

**Por que isso aconteceu?** É a checagem inicial essencial para entender o volume e possíveis problemas (muitos valores nulos, tipos de dados errados).

**O que deveria acontecer:** Se houver um erro de leitura (encoding/CSV mal-formatado), você precisa ajustar os parâmetros de `read_csv`. Também é importante validar se os tipos 'object' deveriam ser numéricos, pois serão tratados na próxima etapa.

In [92]:
df = pd.read_csv(PATH, low_memory=False)

print("\n=== INFO DO DATAFRAME ===")
df.info()


=== INFO DO DATAFRAME ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42675 entries, 0 to 42674
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   title                     42675 non-null  object
 1   rating                    41651 non-null  object
 2   number_of_reviews         41651 non-null  object
 3   bought_in_last_month      39458 non-null  object
 4   current/discounted_price  30926 non-null  object
 5   price_on_variant          42675 non-null  object
 6   listed_price              42675 non-null  object
 7   is_best_seller            42675 non-null  object
 8   is_sponsored              42675 non-null  object
 9   is_couponed               42675 non-null  object
 10  buy_box_availability      28022 non-null  object
 11  delivery_details          30955 non-null  object
 12  sustainability_badges     3408 non-null   object
 13  image_url                 42675 non-null  object


---
## ETAPA 2 — Relatório de nulos e cardinalidade
**O que aconteceu:** Calculamos a contagem e o percentual de valores nulos, não-nulos e a cardinalidade por coluna. Imprimimos o relatório.

**O que está acontecendo:** Você pode ver quais colunas têm mais dados faltando (ex.: preços/entrega) e quais têm pouca variação de valores.

**Por que isso aconteceu?** Essas métricas guiam decisões de limpeza (remover/imputar dados) e ajudam a evitar vieses em análises ou modelos.

**O que deveria acontecer:** Definir regras sobre o que descartar, o que imputar e o que manter mesmo com valores nulos, dependendo do objetivo da sua análise.

In [93]:
null_counts = df.isna().sum()
null_pct = (df.isna().mean() * 100).round(2)

report = pd.DataFrame({
    "column": df.columns,
    "dtype": [str(t) for t in df.dtypes],
    "null_count": [int(null_counts[c]) for c in df.columns],
    "null_%": [float(null_pct[c]) for c in df.columns],
    "non_null_count": [int(len(df) - null_counts[c]) for c in df.columns],
    "unique_values": [int(df[c].nunique(dropna=True)) for c in df.columns],
}).sort_values(by=["null_count","column"], ascending=[False, True])

print("\n=== RELATÓRIO DE NULOS / CARDINALIDADE ===")
print(report)

# (Opcional) Salvar o relatório
# report.to_csv('relatorio_nulos_por_coluna.csv', index=False)


=== RELATÓRIO DE NULOS / CARDINALIDADE ===
                      column   dtype  null_count  null_%  non_null_count  unique_values
12     sustainability_badges  object       39267   92.01            3408             16
10      buy_box_availability  object       14653   34.34           28022              1
4   current/discounted_price  object       11749   27.53           30926           2576
11          delivery_details  object       11720   27.46           30955            298
3       bought_in_last_month  object        3217    7.54           39458             59
14               product_url  object        2069    4.85           40606          40606
2          number_of_reviews  object        1024    2.40           41651           4413
1                     rating  object        1024    2.40           41651             31
15              collected_at  object           0    0.00           42675           1559
13                 image_url  object           0    0.00           42675    

---
## ETAPA 3 — Coerções e features numéricas
**O que aconteceu:** Convertemos campos de texto (preços, avaliações, número de vendas, etc.) em valores numéricos que podem ser usados para análise e modelagem. Criamos colunas derivadas, como o percentual de desconto.

**O que está acontecendo:** O DataFrame agora tem colunas numéricas adicionais que permitem calcular correlações e treinar modelos supervisionados.

**Por que isso aconteceu?** O CSV original continha muitos campos como 'object' (texto), mas modelos de machine learning e correlações exigem valores numéricos. Remover símbolos e padronizar os dados evita falhas de conversão e valores nulos desnecessários.

**O que deveria acontecer:** Validar as estatísticas básicas (ex.: medianas, quartis) e, se ainda houver muitos valores nulos, decidir entre imputá-los, filtrá-los ou criar flags para o modelo.

In [94]:
# ==========================================================================================
# ==== COMENTADO PARA EVITAR CRIAR COLUNAS INUTEIS APENAS PARA ENTENDER RELACIONAMENTOS ====
# ==========================================================================================


# def _to_float_from_money(series: pd.Series) -> pd.Series:
#     """Converte texto monetário para float: remove símbolos, vírgulas e tokens inválidos."""
#     s = (
#         series.astype(str)
#         .str.replace(r'[^0-9.\-]', '', regex=True)
#         .replace({'': np.nan, '.': np.nan, '-': np.nan})
#     )
#     return pd.to_numeric(s, errors='coerce')

# # Preços
# if 'current/discounted_price' in df.columns:
#     df['price_current'] = _to_float_from_money(df['current/discounted_price'])
# if 'listed_price' in df.columns:
#     df['price_listed']  = _to_float_from_money(df['listed_price'])
# if 'price_on_variant' in df.columns:
#     df['price_variant'] = _to_float_from_money(df['price_on_variant'])

# # Desconto (%)
# if {'price_current','price_listed'}.issubset(df.columns):
#     df['discount_pct'] = np.where(
#         (df['price_listed'] > 0) & (df['price_current'] >= 0),
#         (df['price_listed'] - df['price_current'])/df['price_listed'] * 100,
#         np.nan
#     )

# # Rating e reviews (parsing)
# if 'rating' in df.columns:
#     df['rating_num'] = pd.to_numeric(
#         df['rating'].astype(str).str.extract(r'([0-9]+(?:\.[0-9]+)?)', expand=False),
#         errors='coerce'
#     )
# if 'number_of_reviews' in df.columns:
#     df['reviews_num'] = pd.to_numeric(
#         df['number_of_reviews'].astype(str).str.replace(r'[^0-9]', '', regex=True),
#         errors='coerce'
#     )

# # Compras no último mês (extrai número de "100+ bought …")
# if 'bought_in_last_month' in df.columns:
#     df['bought_last_month_num'] = pd.to_numeric(
#         df['bought_in_last_month'].astype(str)
#           .str.extract(r'([0-9,]+)', expand=False)
#           .str.replace(',', ''),
#         errors='coerce'
#     )

# # Flags de entrega (0/1)
# if 'delivery_details' in df.columns:
#     dd = df['delivery_details'].astype(str)
#     df['has_free_delivery'] = dd.str.contains(r'FREE|Free', na=False).astype(int)
#     df['mentions_prime']    = dd.str.contains(r'Prime', na=False).astype(int)

# # Bools comerciais → 0/1
# for col in ['is_best_seller', 'is_sponsored', 'is_couponed', 'buy_box_availability']:
#     if col in df.columns:
#         s = df[col].astype(str).str.lower().str.strip()
#         df[col + '_int'] = s.isin(['true','yes','y','1','available','in stock']).astype(int)

---
## ETAPA 4 — Matriz de Correlação + Heatmap
**O que aconteceu:** Selecionamos as colunas numéricas, calculamos a correlação de Pearson entre elas e geramos um heatmap.

**O que está acontecendo:** Você consegue visualizar as relações lineares entre as variáveis (ex.: `price_current` e `price_listed` provavelmente têm uma alta correlação). Colunas com valores quase constantes ou muitos valores nulos tendem a produzir `NaN`.

**Por que isso aconteceu?** O coeficiente de correlação de Pearson depende da variância e de pares de dados válidos. Se a coluna é constante ou tem muitos valores nulos, a correlação se torna `NaN`.

**O que deveria acontecer:** Use a matriz para identificar redundâncias (evite features colineares) e para guiar a seleção de variáveis para o seu modelo. Considere normalizar ou transformar os dados (ex.: `log1p` em contagens) se necessário.

In [95]:
# ==========================================================================================
# ==== COMENTADO PARA EVITAR CRIAR COLUNAS INUTEIS APENAS PARA ENTENDER RELACIONAMENTOS ====
# ==========================================================================================

# corr_cols = [c for c in [
#     'price_current','price_listed','price_variant','discount_pct',
#     'rating_num','reviews_num','bought_last_month_num',
#     'has_free_delivery','mentions_prime',
#     'is_best_seller_int','is_sponsored_int','is_couponed_int','buy_box_availability_int'
# ] if c in df.columns]

# corr_df = df[corr_cols].copy()
# corr_matrix = corr_df.corr()

# print("\n=== MATRIZ DE CORRELAÇÃO (amostra) ===")
# print(corr_matrix.round(3))

# # (Opcional) Salvar em CSV
# # corr_matrix.to_csv('correlacao_features.csv', index=True)

# # Heatmap simples
# plt.figure(figsize=(10, 8))
# plt.imshow(corr_matrix, interpolation='nearest')
# plt.title('Matriz de Correlação')
# plt.xticks(ticks=range(len(corr_cols)), labels=corr_cols, rotation=90)
# plt.yticks(ticks=range(len(corr_cols)), labels=corr_cols)
# plt.colorbar()
# plt.tight_layout()
# plt.show()

---
## ETAPA 5 — Classificação baseline (hot_seller) + Matriz de Confusão
**O que aconteceu:** Criamos o target `hot_seller` (produtos com >= 100 vendas no mês) e treinamos uma Regressão Logística como modelo baseline. Avaliamos o modelo usando um conjunto de teste estratificado e exibimos a matriz de confusão e o relatório de classificação.

**O que está acontecendo:** O recall da classe `hot` (classe 1) é muito alto (~0.94), mas o da classe `not hot` (classe 0) é baixo (~0.37), o que gera muitos falsos positivos (itens previstos como 1, mas que são 0 na realidade).

**Por que isso aconteceu?** O desbalanceamento dos dados (a classe 1 é maior) e o limite de decisão padrão de 0.5 empurram o classificador para prever mais 1s. Além disso, as features podem capturar bem os sinais de `hot`, mas talvez faltem dados suficientes para distinguir de forma confiável os itens `not hot`.

**O que deveria acontecer:** Você pode ajustar o limite de decisão do classificador (usando `predict_proba`), usar o parâmetro `class_weight='balanced'`, ou incluir novas features (como TF-IDF do título do produto). Experimente também modelos baseados em árvores, como LightGBM ou XGBoost, que costumam lidar melhor com dados desbalanceados. Por fim, revise o tratamento de valores nulos e o parsing dos campos para garantir que todas as variáveis sejam robustas e informativas.

In [96]:
# ==========================================================================================
# ==== COMENTADO PARA EVITAR CRIAR COLUNAS INUTEIS APENAS PARA ENTENDER RELACIONAMENTOS ====
# ==========================================================================================

# # Definição do alvo binário
# HOT_THRESHOLD = 100  # ajuste conforme estratégia de negócio

# if 'bought_last_month_num' not in df.columns and 'bought_in_last_month' in df.columns:
#     df['bought_last_month_num'] = pd.to_numeric(
#         df['bought_in_last_month'].astype(str)
#           .str.extract(r'([0-9,]+)', expand=False)
#           .str.replace(',', ''),
#         errors='coerce'
#     )

# df['hot_seller'] = (df['bought_last_month_num'] >= HOT_THRESHOLD).astype(int)

# # Features (foco nas de maior disponibilidade/robustez)
# feat_cols = [c for c in [
#     'price_current','price_listed','discount_pct',
#     'rating_num','reviews_num',
#     'has_free_delivery','mentions_prime'
# ] if c in df.columns]

# # Filtra linhas com NaN nas features/target
# data = df.dropna(subset=feat_cols + ['hot_seller']).copy()
# X = data[feat_cols].values
# y = data['hot_seller'].values

# # Hold-out estratificado
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.25, random_state=42, stratify=y
# )

# # Modelo baseline
# clf = LogisticRegression(max_iter=500)
# clf.fit(X_train, y_train)

# # Predição
# y_pred = clf.predict(X_test)

# # Matriz de confusão e relatório
# cm = confusion_matrix(y_test, y_pred, labels=[0, 1])
# print("\n=== MATRIZ DE CONFUSÃO ===")
# print(cm)
# print("\n=== CLASSIFICATION REPORT ===")
# print(classification_report(y_test, y_pred, digits=3))

# # Visualização
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['not hot','hot'])
# disp.plot()
# plt.title('Matriz de Confusão – Classificador baseline (hot_seller)')
# plt.tight_layout()
# plt.show()

In [97]:
# Analisar a coluna 'sustainability_badges' para entender a estrutura dos valores
print("\n=== Análise da coluna 'sustainability_badges' ===")

# Contagem de valores únicos (excluindo NaN, que já foi tratado como 0)
print("\nContagem de cada valor único (excluindo 0/NaN):")
print(df['sustainability_badges'].value_counts(dropna=True))

# Verificar se há valores que indicam múltiplos badges (baseado na lista de IDs)
print("\nValores de 'sustainability_badges' que podem indicar múltiplos selos:")
# Filtrar os valores que contêm "more" ou "+" na descrição original (usando o mapa id_to_label)
multi_badge_ids = [id for id, label in id_to_label.items() if isinstance(label, str) and ('more' in label.lower() or '+' in label)]
multi_badge_labels = [label for id, label in id_to_label.items() if isinstance(label, str) and ('more' in label.lower() or '+' in label)]

print(f"IDs que podem representar múltiplos selos: {multi_badge_ids}")
print(f"Labels correspondentes: {multi_badge_labels}")

# Filtrar o dataframe para mostrar exemplos de linhas com esses IDs
examples_multi = df[df['sustainability_badges'].isin(multi_badge_ids)]

if not examples_multi.empty:
    print("\nExemplos de linhas com múltiplos selos (baseado nos IDs identificados):")
    display(examples_multi[['sustainability_badges', BADGE_COL]].head()) # Mostra o ID e o valor original

else:
    print("\nNenhuma entrada encontrada que corresponda aos IDs que podem indicar múltiplos selos.")

# Contagem total de valores não-zero (considerados com algum badge)
print(f"\nTotal de produtos com algum selo (não NaN): {df['sustainability_badges'][df['sustainability_badges'] != 0].count()}")


=== Análise da coluna 'sustainability_badges' ===

Contagem de cada valor único (excluindo 0/NaN):
sustainability_badges
Small Business                    1341
Carbon impact                      769
Works with Alexa                   425
Energy efficiency                  262
Alexa Built-in                     184
Manufacturing practices            148
Energy efficiency +3 more          121
Energy efficiency +1 more           93
Forestry practices                  18
Safer chemicals +2 more             18
Recycled materials                   7
Recycled materials +2 more           7
Safer chemicals +1 more              6
Recycled materials +3 more           4
Made in Italy                        4
1 sustainability certification       1
Name: count, dtype: int64

Valores de 'sustainability_badges' que podem indicar múltiplos selos:
IDs que podem representar múltiplos selos: [4, 10, 12, 13, 14, 15]
Labels correspondentes: ['Safer chemicals +1 more', 'Energy efficiency +3 more', 'Safer ch

# Tratamento dos dados / Normalização dos dados

1.   Adicionar labels que substituam as classificações por ids da coluna "sustainability_badges", onde o Nan sempre será 0.0

2. Converter valores dos intervalos de rating de string para float (real), onde o Nan sempre será 0.0

3. Adicionar labels que substituam as classificações por valores relacionados a quantidade da coluna "bought_in_last_month", onde o Nan e valores fora do intervalo sempre seram 0.0

4. Converter valores da coluna "current/discounted_price" de string para float (real), onde o Nan sempre será 0.0

5. Adicionar labels que substituam as classificações por valores relacionados a quantidade da coluna "price_on_variant", onde o Nan e valores fora do intervalo sempre seram 0.0



In [98]:
# ==================================
# Normalização da coluna sustainability_badges
# ==================================

# === Mapa de IDs para sustainability_badges ===
# 0: NaN
# 1: Small Business
# 2: Carbon impact
# 3: Energy efficiency
# 4: Safer chemicals +1 more
# 5: 1 sustainability certification
# 6: Works with Alexa
# 7: Manufacturing practices
# 8: Forestry practices
# 9: Alexa Built-in
# 10: Energy efficiency +3 more
# 11: Recycled materials
# 12: Safer chemicals +2 more
# 13: Energy efficiency +1 more
# 14: Recycled materials +3 more
# 15: Recycled materials +2 more
# 16: Made in Italy

# --- Converter sustainability_badges (strings) -> IDs numéricos (in-place) ---
# Localiza a coluna mesmo que exista espaço extra no nome
badge_cols = [c for c in df.columns if c.strip().lower() == "sustainability_badges"]
if not badge_cols:
    raise KeyError("Coluna 'sustainability_badges' não encontrada.")
BADGE_COL = badge_cols[0]

# Normaliza espaços; preserva NaN nesta etapa
df[BADGE_COL] = df[BADGE_COL].astype("string").str.strip()

# Lista de valores únicos esperados (sem o NaN):
prefered_order = [
    'Carbon impact',
    'Energy efficiency',
    'Safer chemicals +1 more',
    'Small Business',
    '1 sustainability certification',
    'Works with Alexa',
    'Manufacturing practices',
    'Forestry practices',
    'Alexa Built-in',
    'Energy efficiency +3 more',
    'Recycled materials',
    'Safer chemicals +2 more',
    'Energy efficiency +1 more',
    'Recycled materials +3 more',
    'Recycled materials +2 more',
    'Made in Italy',
]

# Pega os rótulos que realmente existem no dataframe (exclui NaN)
present = [x for x in df[BADGE_COL].dropna().unique().tolist()]
# Ordena deterministicamente; extras (se existirem) vão para o final
ordered = [x for x in prefered_order if x in present] + [x for x in present if x not in prefered_order]

# IDs fixos
fixed_ids = {
    'Small Business': 1,
    'Carbon impact': 2,
}

# Constrói o mapeamento label -> id (respeitando os fixos e depois enumerando o resto)
used = set(fixed_ids.keys())
start_id = max(fixed_ids.values(), default=0) + 1
auto_labels = [x for x in ordered if x not in used]
auto_ids = {label: i for i, label in enumerate(auto_labels, start=start_id)}

label_to_id = {**fixed_ids, **auto_ids}  # ex.: {'Small Business':1, 'Carbon impact':2, ...}

# Aplica a substituição IN-PLACE e faz NaN / não mapeados -> 0
df[BADGE_COL] = df[BADGE_COL].map(label_to_id).fillna(0).astype("Int64")

# Gera o mapa id -> label (inclui 0: NaN) para conferência
id_to_label = {0: "NaN", **{v: k for k, v in label_to_id.items()}}

print("\n=== Mapa de IDs para sustainability_badges ===")
for k in sorted(id_to_label):
    print(f"{k}: {id_to_label[k]}")

# print("\n=== 1) CONTAGEM DE VALORES ÚNICOS EM 'sustainability_badges' ===")
# print(df['sustainability_badges'].value_counts(dropna=False))

# print("\n=== VALORES ÚNICOS EM 'sustainability_badges' ===")
# print(df['sustainability_badges'].unique())


=== Mapa de IDs para sustainability_badges ===
0: NaN
1: Small Business
2: Carbon impact
3: Energy efficiency
4: Safer chemicals +1 more
5: 1 sustainability certification
6: Works with Alexa
7: Manufacturing practices
8: Forestry practices
9: Alexa Built-in
10: Energy efficiency +3 more
11: Recycled materials
12: Safer chemicals +2 more
13: Energy efficiency +1 more
14: Recycled materials +3 more
15: Recycled materials +2 more
16: Made in Italy


In [99]:

# ==================================
# Normalização da coluna rating
# ==================================


# Localiza a coluna de rating sem renomear (suporta "rating" ou "rating " etc.)
rating_cols = [c for c in df.columns if c.strip().lower() == "rating"]
if not rating_cols:
    raise KeyError("Coluna de rating não encontrada (procurei por 'rating').")
RATING_COL = rating_cols[0]  # usamos o nome exato existente (com ou sem espaços)

# Extrai o número e converte para float; valores não parseados viram 0.0
df[RATING_COL] = (
    pd.to_numeric(
        df[RATING_COL]
          .astype(str)
          .str.replace(',', '.', regex=False)                     # vírgula -> ponto
          .str.extract(r'([0-9]+(?:\.[0-9]+)?)', expand=False),   # "4.6 out of 5 stars" -> "4.6"
        errors='coerce'                                           # falhas -> NaN
    )
    .fillna(0.0)                                                  # NaN -> 0.0
    .astype(float)                                                # garante float nativo
)

# print("\n=== 2) CONTAGEM DE VALORES ÚNICOS EM 'rating' ===")
# print(df['rating'].value_counts(dropna=False))


# print("\n=== VALORES ÚNICOS EM 'rating' ===")
# print(df['rating'].unique())

In [100]:

# =======================================
# Normalização da bought_in_last_month
# =======================================

# === Mapa de valores para bought_in_last_month ===
# Todos os valores normalizados seguem o seguinte padrão
# 6K+ bought in past month → 6000
# 300+ bought in past month → 300
# ($4,813.00$4,813.00/100 cm) / ($433.33$433.33/count) / List Price: / etc. → 0 (para manter o padrão numérico da coluna)


# --- Converter "bought_in_last_month" p/ inteiros e normalizar ruído ---------
# Localiza a coluna mesmo com espaços extras
bcols = [c for c in df.columns if c.strip().lower() == "bought_in_last_month"]
if not bcols:
    raise KeyError("Coluna 'bought_in_last_month' não encontrada.")
B_COL = bcols[0]

# Trabalha como string; preserva NaN
s = df[B_COL].astype("string")

# Captura padrões válidos de compra: "300+ bought in past/last month", "6K+ bought...", "100K+..."
pat = r'(?i)\b(\d+(?:\.\d+)?)\s*([KMB])?\s*\+\s*bought\s+in\s+(?:past|last)\s+month\b'
m = s.str.extract(pat, expand=True)

num = pd.to_numeric(m[0], errors="coerce")
mult = m[1].str.upper().map({"K": 1_000, "M": 1_000_000, "B": 1_000_000_000}).fillna(1)
parsed = (num * mult).round()

# === Normalização dos "não-contagens"
# Tudo que NÃO bateu no padrão acima vira 0 (mantém a coluna numérica e “no padrão”)
# Exemplos: "($4,813.00$4,813.00/100 cm)", "($433.33$433.33/count)", "List Price:", "Typical:", etc.
df[B_COL] = parsed.fillna(0).astype("float")



# print("\n=== 3) CONTAGEM DE VALORES ÚNICOS EM 'bought_in_last_month' ===")
# print(df['bought_in_last_month'].value_counts(dropna=False))

# print("\n=== VALORES ÚNICOS EM 'bought_in_last_month' ===")
# print(df['bought_in_last_month'].unique())


In [101]:
# ==========================================
# Normalização de current/discounted_price
# ==========================================

# Localiza a coluna mesmo que tenha espaços extras no nome
price_cols = [c for c in df.columns if c.strip().lower() == "current/discounted_price"]
if not price_cols:
    raise KeyError("Coluna 'current/discounted_price' não encontrada.")
PRICE_COL = price_cols[0]

df[PRICE_COL] = (
    pd.to_numeric(
        df[PRICE_COL]
          .astype("string")
          # remove símbolos/moedas e qualquer coisa que não seja dígito, ponto, vírgula ou sinal
          .str.replace(r"[^\d,\.\-]", "", regex=True)
          # remove vírgulas de milhar (mantendo ponto decimal, p.ex. '1,579.99' -> '1579.99')
          .str.replace(",", "", regex=False)
          # limpa casos patológicos como apenas '.' ou '-' (virarão NaN e depois 0.0)
          .str.replace(r"^\.$|^-$|^$", "", regex=True),
        errors="coerce"
    )
    .fillna(0.0)
    .astype(float)
)


# print("\n=== 4) CONTAGEM DE VALORES ÚNICOS EM 'current/discounted_price' ===")
# print(df['current/discounted_price'].value_counts(dropna=False))

# print("\n=== VALORES ÚNICOS EM 'current/discounted_price' ===")
# print(df['current/discounted_price'].unique())


In [102]:
# ==========================================
# Normalização da coluna "price_on_variant"
# ==========================================

# === Mapa de valores para price_on_variant ===
#basic variant price: $9.99 → 9.99
#basic variant price: $119.95 → 119.95
#basic variant price: $169.99 → 169.99
#basic variant price: $90.00 off coupon applied → 90.00
#basic variant price: 16 Count (Pack of 1) → 0.0 (não tem moeda)
#basic variant price: nan → 0.0

# Localiza a coluna mesmo que tenha espaços extras no nome
pcols = [c for c in df.columns if c.strip().lower() == "price_on_variant"]
if not pcols:
    raise KeyError("Coluna 'price_on_variant' não encontrada.")
PVAR_COL = pcols[0]

# Trabalha como string, sem alterar o nome da coluna
s = df[PVAR_COL].astype("string")

# Regex:
# - aceita símbolos de moeda: $, €, £, ¥, ₹
# - também aceita códigos: USD, EUR, GBP, JPY, INR (case-insensitive)
# - captura a parte numérica com milhares (vírgulas) e decimais
pat = r'(?i)(?:[\$\€\£\¥\₹]\s*|(?:USD|EUR|GBP|JPY|INR)\s*)([0-9]+(?:,[0-9]{3})*(?:\.[0-9]+)?)'

# Extrai o 1º preço encontrado; casos sem match ficam NaN
m = s.str.extract(pat, expand=False)

# Converte para float (remove vírgulas de milhar), NaN -> 0.0
df[PVAR_COL] = (
    pd.to_numeric(m.str.replace(",", "", regex=False), errors="coerce")
      .fillna(0.0)
      .astype(float)
)


# print("\n=== 5) CONTAGEM DE VALORES ÚNICOS EM 'price_on_variant' ===")
# print(df['price_on_variant'].value_counts(dropna=False))

# print("\n=== VALORES ÚNICOS EM 'price_on_variant' ===")
# print(df['price_on_variant'].unique())

In [103]:
print("\n=== 6) CONTAGEM DE VALORES ÚNICOS EM 'listed_price' ===")
print(df['listed_price'].value_counts(dropna=False))

print("\n=== VALORES ÚNICOS EM 'listed_price' ===")
print(df['listed_price'].unique())

print('\n\n\n\n\n\n\n\n')
df.info()


=== 6) CONTAGEM DE VALORES ÚNICOS EM 'listed_price' ===
listed_price
No Discount    30364
$79.99           500
$29.99           406
$16.61           360
$23.26           283
               ...  
$42.50             1
$2,999.99          1
$16.97             1
$71.87             1
$14.33             1
Name: count, Length: 911, dtype: int64

=== VALORES ÚNICOS EM 'listed_price' ===
['$159.00' '$15.99' '$349.00' 'No Discount' '$19.98' '$14.33' '$17.99'
 '$21.89' '$20.00' '$48.99' '$19.99' '$38.98' '$14.39' '$169.99' '$29.99'
 '$219.99' '$15.00' '$24.99' '$32.72' '$71.87' '$18.99' '$21.44' '$199.99'
 '$99.95' '$79.99' '$49.99' '$33.99' '$39.99' '$89.99' '$119.97' '$35.64'
 '$24.58' '$69.99' '$249.99' '$27.99' '$6.29' '$239.99' '$229.99'
 '$169.00' '$32.29' '$34.99' '$348.00' '$65.99' '$5.84' '$176.99' '$96.99'
 '$11.87' '$43.29' '$64.99' '$159.95' '$45.79' '$109.99' '$8.99' '$32.99'
 '$149.99' '$25.99' '$339.99' '$99.99' '$35.99' '$449.00' '$76.99'
 '$165.00' '$13.33' '$7.79' '$30.00' '$129

In [104]:
# ==========================================
# Normalização da coluna "number_of_reviews"
# ==========================================

reviews_cols = [c for c in df.columns if c.strip().lower() == "number_of_reviews"]
if not reviews_cols:
    raise KeyError("Coluna 'number_of_reviews' não encontrada.")
REVIEWS_COL = reviews_cols[0]

df[REVIEWS_COL] = (
    pd.to_numeric(
        df[REVIEWS_COL]
          .astype("string")
          .str.replace(",", "", regex=False), # Remove vírgulas de milhar
        errors="coerce" # Converte falhas para NaN
    )
    .fillna(0.0) # Preenche NaN com 0.0
    .astype(float) # Garante tipo float nativo
)

print("\n=== Normalização da coluna 'number_of_reviews' concluída ===")
print(df[REVIEWS_COL].head())
print("\n=== df.info() após a normalização de 'number_of_reviews' ===")
df.info()


=== Normalização da coluna 'number_of_reviews' concluída ===
0      375.0
1     2457.0
2     3044.0
3    35882.0
4    28988.0
Name: number_of_reviews, dtype: float64

=== df.info() após a normalização de 'number_of_reviews' ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42675 entries, 0 to 42674
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   title                     42675 non-null  object 
 1   rating                    42675 non-null  float64
 2   number_of_reviews         42675 non-null  float64
 3   bought_in_last_month      42675 non-null  float64
 4   current/discounted_price  42675 non-null  float64
 5   price_on_variant          42675 non-null  float64
 6   listed_price              42675 non-null  object 
 7   is_best_seller            42675 non-null  object 
 8   is_sponsored              42675 non-null  object 
 9   is_couponed               42675 non-null  object 
 1

In [105]:
df.head()

,title,rating,number_of_reviews,bought_in_last_month,current/discounted_price,price_on_variant,listed_price,is_best_seller,is_sponsored,is_couponed,buy_box_availability,delivery_details,sustainability_badges,image_url,product_url,collected_at
0,BOYA BOYALINK 2 Wireless Lavalier Microphone f...,4.6,375.0,300.0,89.68,0.00,$159.00,No Badge,Sponsored,Save 15% with coupon,Add to cart,"Delivery Mon, Sep 1",2,https://m.media-amazon.com/images/I/71pAqiVEs3...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
1,"LISEN USB C to Lightning Cable, 240W 4 in 1 Ch...",4.3,2457.0,6000.0,9.99,0.00,$15.99,No Badge,Sponsored,No Coupon,Add to cart,"Delivery Fri, Aug 29",0,https://m.media-amazon.com/images/I/61nbF6aVIP...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
2,"DJI Mic 2 (2 TX + 1 RX + Charging Case), Wirel...",4.6,3044.0,2000.0,314.00,0.00,$349.00,No Badge,Sponsored,No Coupon,Add to cart,"Delivery Mon, Sep 1",0,https://m.media-amazon.com/images/I/61h78MEXoj...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
3,"Apple AirPods Pro 2 Wireless Earbuds, Active N...",4.6,35882.0,10000.0,0.00,162.24,No Discount,Best Seller,Organic,No Coupon,NaN,NaN,0,https://m.media-amazon.com/images/I/61SUj2aKoE...,/Apple-Cancellation-Transparency-Personalized-...,2025-08-21 11:14:29
4,Apple AirTag 4 Pack. Keep Track of and find Yo...,4.8,28988.0,10000.0,0.00,72.74,No Discount,No Badge,Organic,No Coupon,NaN,NaN,0,https://m.media-amazon.com/images/I/61bMNCeAUA...,/Apple-MX542LL-A-AirTag-Pack/dp/B0D54JZTHY/ref...,2025-08-21 11:14:29


# Task
Tratar os valores ausentes nas colunas `current/discounted_price` e `listed_price` e converter a coluna `delivery_details` para datetime.

## Tratar valores ausentes em `current/discounted price`

### Subtask:
Preencher os valores ausentes na coluna `current/discounted_price` com os valores da coluna `price_on_variant` onde `current/discounted_price` estiver faltando. Certifique-se de que ambas as colunas estejam no formato numérico correto antes de realizar o preenchimento.


**Reasoning**:
Fill missing values in `current/discounted_price` with values from `price_on_variant` where `current/discounted_price` is missing.



In [106]:
# Ensure both columns are numeric (float)
df['current/discounted_price'] = df['current/discounted_price'].astype(float)
df['price_on_variant'] = df['price_on_variant'].astype(float)

# Identify rows where 'current/discounted_price' is missing (NaN or 0.0)
# Since we filled NaNs with 0.0 in previous steps, we check for 0.0
mask_missing_price = df['current/discounted_price'] == 0.0

# Fill missing 'current/discounted_price' values with 'price_on_variant' where available
df.loc[mask_missing_price, 'current/discounted_price'] = df.loc[mask_missing_price, 'price_on_variant']

print("\n=== Preenchimento de valores ausentes em 'current/discounted_price' concluído ===")
print(df[['current/discounted_price', 'price_on_variant']].head())
print("\n=== df.info() após o preenchimento ===")
df.info()


=== Preenchimento de valores ausentes em 'current/discounted_price' concluído ===
   current/discounted_price  price_on_variant
0                     89.68              0.00
1                      9.99              0.00
2                    314.00              0.00
3                    162.24            162.24
4                     72.74             72.74

=== df.info() após o preenchimento ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42675 entries, 0 to 42674
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   title                     42675 non-null  object 
 1   rating                    42675 non-null  float64
 2   number_of_reviews         42675 non-null  float64
 3   bought_in_last_month      42675 non-null  float64
 4   current/discounted_price  42675 non-null  float64
 5   price_on_variant          42675 non-null  float64
 6   listed_price              42675 non-null  objec

## Tratar valores ausentes em `listed price`

### Subtask:
Preencher os valores ausentes (que são 0.0 após a normalização anterior, representando "No Discount") na coluna `listed_price` com os valores da coluna `current/discounted_price`. Novamente, verifique se ambas as colunas estão numéricas.


**Reasoning**:
The previous step already handled the `current/discounted_price` column. The current subtask focuses on `listed_price` and ensuring both are numeric before filling `listed_price`. The provided history indicates that 'No Discount' in `listed_price` might have resulted in 0.0 after previous normalizations, which is the value we need to identify and fill.



In [107]:
# 1. Convert the listed_price column to a numeric type (float), coercing errors to NaN and filling resulting NaNs with 0.0.
# This assumes "No Discount" was handled in a previous step to result in a value that will be treated as missing (like NaN or 0.0) for the purpose of this fill.
# We'll re-apply a robust numeric conversion just in case.
listed_price_cols = [c for c in df.columns if c.strip().lower() == "listed_price"]
if not listed_price_cols:
    raise KeyError("Coluna 'listed_price' não encontrada.")
LISTED_PRICE_COL = listed_price_cols[0]

df[LISTED_PRICE_COL] = (
    pd.to_numeric(
        df[LISTED_PRICE_COL]
          .astype("string")
          .str.replace(r"[^\d,\.\-]", "", regex=True) # remove symbols except digits, comma, period, hyphen
          .str.replace(",", "", regex=False) # remove thousands separator comma
          .str.replace(r"^\.$|^-$|^$", "", regex=True), # clean up isolated period or hyphen
        errors="coerce" # Convert failures to NaN
    )
    .fillna(0.0) # Fill NaN with 0.0 (representing 'No Discount' or parse errors)
    .astype(float) # Ensure native float type
)


# 2. Ensure the current/discounted_price column is of a numeric type (float).
# This was handled in the previous subtask, but we'll ensure it again.
current_price_cols = [c for c in df.columns if c.strip().lower() == "current/discounted_price"]
if not current_price_cols:
    raise KeyError("Coluna 'current/discounted_price' não encontrada.")
CURRENT_PRICE_COL = current_price_cols[0]
df[CURRENT_PRICE_COL] = df[CURRENT_PRICE_COL].astype(float)


# 3. Identify rows where listed_price is 0.0, as these are the values to be filled.
mask_missing_listed_price = df[LISTED_PRICE_COL] == 0.0

# 4. For the identified rows, replace the 0.0 values in listed_price with the corresponding values from the current/discounted_price column.
# Only fill if current/discounted_price is not also 0.0 (to avoid filling 0 with 0)
fill_mask = mask_missing_listed_price & (df[CURRENT_PRICE_COL] != 0.0)
df.loc[fill_mask, LISTED_PRICE_COL] = df.loc[fill_mask, CURRENT_PRICE_COL]


# 5. Print the head of the DataFrame showing the listed_price and current/discounted_price columns to visually inspect the changes.
print("\n=== Visual inspection of 'listed_price' and 'current/discounted_price' after filling ===")
display(df[[LISTED_PRICE_COL, CURRENT_PRICE_COL]].head())

# 6. Print the df.info() to verify the data types and non-null counts of the affected columns.
print("\n=== df.info() after filling 'listed_price' ===")
df.info()


=== Visual inspection of 'listed_price' and 'current/discounted_price' after filling ===


,listed_price,current/discounted_price
0,159.00,89.68
1,15.99,9.99
2,349.00,314.00
3,162.24,162.24
4,72.74,72.74



=== df.info() after filling 'listed_price' ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42675 entries, 0 to 42674
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   title                     42675 non-null  object 
 1   rating                    42675 non-null  float64
 2   number_of_reviews         42675 non-null  float64
 3   bought_in_last_month      42675 non-null  float64
 4   current/discounted_price  42675 non-null  float64
 5   price_on_variant          42675 non-null  float64
 6   listed_price              42675 non-null  float64
 7   is_best_seller            42675 non-null  object 
 8   is_sponsored              42675 non-null  object 
 9   is_couponed               42675 non-null  object 
 10  buy_box_availability      28022 non-null  object 
 11  delivery_details          30955 non-null  object 
 12  sustainability_badges     42675 non-null  Int64  
 13  image_url    

In [109]:
print("\n=== df.info() after all normalizations and fillings ===")
df.info()

print("\n=== df.head() after all normalizations and fillings ===")
display(df.head())


=== df.info() after all normalizations and fillings ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42675 entries, 0 to 42674
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   title                     42675 non-null  object 
 1   rating                    42675 non-null  float64
 2   number_of_reviews         42675 non-null  float64
 3   bought_in_last_month      42675 non-null  float64
 4   current/discounted_price  42675 non-null  float64
 5   price_on_variant          42675 non-null  float64
 6   listed_price              42675 non-null  float64
 7   is_best_seller            42675 non-null  object 
 8   is_sponsored              42675 non-null  object 
 9   is_couponed               42675 non-null  object 
 10  buy_box_availability      28022 non-null  object 
 11  delivery_details          30955 non-null  object 
 12  sustainability_badges     42675 non-null  Int64  
 13  imag

,title,rating,number_of_reviews,bought_in_last_month,current/discounted_price,price_on_variant,listed_price,is_best_seller,is_sponsored,is_couponed,buy_box_availability,delivery_details,sustainability_badges,image_url,product_url,collected_at
0,BOYA BOYALINK 2 Wireless Lavalier Microphone f...,4.6,375.0,300.0,89.68,0.00,159.00,No Badge,Sponsored,Save 15% with coupon,Add to cart,"Delivery Mon, Sep 1",2,https://m.media-amazon.com/images/I/71pAqiVEs3...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
1,"LISEN USB C to Lightning Cable, 240W 4 in 1 Ch...",4.3,2457.0,6000.0,9.99,0.00,15.99,No Badge,Sponsored,No Coupon,Add to cart,"Delivery Fri, Aug 29",0,https://m.media-amazon.com/images/I/61nbF6aVIP...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
2,"DJI Mic 2 (2 TX + 1 RX + Charging Case), Wirel...",4.6,3044.0,2000.0,314.00,0.00,349.00,No Badge,Sponsored,No Coupon,Add to cart,"Delivery Mon, Sep 1",0,https://m.media-amazon.com/images/I/61h78MEXoj...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
3,"Apple AirPods Pro 2 Wireless Earbuds, Active N...",4.6,35882.0,10000.0,162.24,162.24,162.24,Best Seller,Organic,No Coupon,NaN,NaN,0,https://m.media-amazon.com/images/I/61SUj2aKoE...,/Apple-Cancellation-Transparency-Personalized-...,2025-08-21 11:14:29
4,Apple AirTag 4 Pack. Keep Track of and find Yo...,4.8,28988.0,10000.0,72.74,72.74,72.74,No Badge,Organic,No Coupon,NaN,NaN,0,https://m.media-amazon.com/images/I/61bMNCeAUA...,/Apple-MX542LL-A-AirTag-Pack/dp/B0D54JZTHY/ref...,2025-08-21 11:14:29
